# Bloomberg BQuant Webinar Series: Intro to BQuant

This is a companion notebook to the Bloomberg [Intro to BQuant](https://blinks.bloomberg.com/screens/PLYR%20VOD%20329737029) webinar (annotated).  

In [1]:
#Libary to Fetch Data from Bloombg
import bql

#Libraries for Data Visualization
from ipywidgets import Button, VBox, HBox, Layout
from IPython.display import display
from bqwidgets import TickerAutoComplete, DataGrid

#Data Manipulation
import pandas as pd

In [2]:
#Function to convert the result of a BQL query to a normal pandas dataframe
def merge_dfs(response):
    all_dfs = [item.df()[item.name] for item in response]
    return pd.concat(all_dfs, axis=1)

In [3]:
# Start service to fetch data
bq = bql.Service()

In [4]:
#Create & initialize input box, Run button, and display grid
input_ticker = TickerAutoComplete(value = 'UKX Index', yellow_keys = ['Index'], layout=Layout(width='150px'))
run_button = Button(description = 'Run', layout=Layout(width='100px'))
result_box = VBox([], layout=Layout(width='500px'))

In [5]:
#Function to fetch index constituents, related fields, and rename columns
def get_data(caller):
    universe = bq.univ.members(input_ticker.value)
    fields= {'Name' : bq.data.name(),
             'Price' : bq.data.px_last(),
             'P/E' : bq.data.pe_ratio(fill='prev')}
    request = bql.Request(universe, fields)
    resp = bq.execute(request)
    column_defs = [{'headerName' : 'Ticker', 'field':'ID', 'width': 150},
               {'headerName' : 'Name', 'field':'Name', 'width': 150},
               {'headerName' : 'Price', 'field':'Price', 'width': 80},
               {'headerName' : 'P/E', 'field':'P/E', 'width': 80}]
    data = merge_dfs(resp)
    dgrid = DataGrid(data=data.round(2).reset_index(), column_defs=column_defs)
    result_box.children = [dgrid]

In [6]:
#fetch the data when button is clicked
run_button.on_click(get_data)

In [7]:
#Connect the widgets and link
ui = VBox(layout=Layout(width='600px'))
ui.children = [HBox([input_ticker, run_button]), result_box]
display(ui)

# What just happened

In [8]:
universe = bq.univ.members('INDU Index')
fields= {'Name' : bq.data.name(),
         'Price' : bq.data.px_last(),
         'P/E' : bq.data.pe_ratio(fill='prev')}

request = bql.Request(universe, fields)
resp = bq.execute(request)

In [9]:
resp[1].df().head(7)

,Name
ID,
AXP UN Equity,American Express Co
VZ UN Equity,Verizon Communications Inc
BA UN Equity,Boeing Co/The
CAT UN Equity,Caterpillar Inc
JPM UN Equity,JPMorgan Chase & Co
CVX UN Equity,Chevron Corp
KO UN Equity,Coca-Cola Co/The


In [10]:
column_defs = [{'headerName' : 'Ticker', 'field':'ID', 'width': 150},
               {'headerName' : 'Name', 'field':'Name', 'width': 150},
               {'headerName' : 'Price', 'field':'Price', 'width': 80},
               {'headerName' : 'P/E', 'field':'P/E', 'width': 80}]

data = merge_dfs(resp)

In [11]:
data.head(7)

,P/E,Name,Price
ID,,,
AXP UN Equity,16.397562,American Express Co,134.460007
VZ UN Equity,12.197879,Verizon Communications Inc,58.650002
BA UN Equity,NaN,Boeing Co/The,342.820007
CAT UN Equity,12.651860,Caterpillar Inc,139.720001
JPM UN Equity,13.209512,JPMorgan Chase & Co,137.880005
CVX UN Equity,18.276105,Chevron Corp,111.660004
KO UN Equity,29.182494,Coca-Cola Co/The,59.610001


In [12]:
dgrid = DataGrid(data=data.round(2).reset_index(), column_defs=column_defs)
dgrid

DataGrid(column_defs=[{'field': 'ID', 'width': 150, 'headerName': 'Ticker'}, {'field': 'Name', 'width': 150, '…